# Flights data exploration
## by Jean-Luc COSSI

## Preliminary Wrangling

> We use Python visualization libraries to systematically explore "Reporting Carrier On-Time Performance" for several years. Those data files have been selected from http://stat-computing.org/dataexpo/2009/the-data.html.  
A detailed presentation of the data is available here https://www.transtats.bts.gov/Fields.asp?Table_ID=236

In [3]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

> Load in the dataset and describe its properties. We motivate our exploration goals through this section.

In [4]:
# load data files
df_1987 = pd.read_csv('data/1988.csv')
df_1989 = pd.read_csv('data/1989.csv')
df_1990 = pd.read_csv('data/1990.csv')
df_1991 = pd.read_csv('data/1991.csv')
df_1992 = pd.read_csv('data/1992.csv')

In [5]:
# concat all into one dafaframe
df_flight = pd.concat([df_1987, df_1989, df_1990, df_1991, df_1992], ignore_index=True)

In [6]:
# Get the dataframe size
df_flight.shape

(25683271, 29)

In [7]:
# Get a view on some values
df_flight.sample(5)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
24017811,1992,9,25,5,2214.0,1905,2249.0,1940,US,762,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
17977728,1991,6,11,2,1602.0,1600,2218.0,2203,AA,1292,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
10074718,1989,12,18,1,1810.0,1810,1946.0,1934,DL,1253,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
17796839,1991,6,27,4,2001.0,2001,2056.0,2115,UA,647,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
13286245,1990,7,28,6,828.0,830,942.0,949,CO,1173,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [8]:
# Get the columns types
df_flight.dtypes

Year                   int64
Month                  int64
DayofMonth             int64
DayOfWeek              int64
DepTime              float64
CRSDepTime             int64
ArrTime              float64
CRSArrTime             int64
UniqueCarrier         object
FlightNum              int64
TailNum              float64
ActualElapsedTime    float64
CRSElapsedTime         int64
AirTime              float64
ArrDelay             float64
DepDelay             float64
Origin                object
Dest                  object
Distance             float64
TaxiIn               float64
TaxiOut              float64
Cancelled              int64
CancellationCode     float64
Diverted               int64
CarrierDelay         float64
WeatherDelay         float64
NASDelay             float64
SecurityDelay        float64
LateAircraftDelay    float64
dtype: object

### What is the structure of your dataset?

> Your answer here!

### What is/are the main feature(s) of interest in your dataset?

> Your answer here!

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

> Your answer here!

## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

> Make sure that, after every plot or related series of plots, that you
include a Markdown cell with comments about what you observed, and what
you plan on investigating next.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!